<a href="https://colab.research.google.com/github/tipiorgup/Tutorial_IDP_SAPs/blob/main/Simulation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import sys


In [ ]:
!git clone https://github.com/tipiorgup/Tutorial_IDP_SAPs.git
!pip install openmm
!pip install MDAnalysis 